# Generate CMA using tick data service - C#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *cumulative moving average - CMA* from a tick data service.

### Inputs/outputs
Cumulative moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

CMA is the average of all the data points.

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick data from the hosted service. The queried endpoint in this script are:
* *TickCmaService *: to directly retrieve tick cma data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run CMA sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *CMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Cumulative moving average request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 CMA parameters

In [ ]:
var field = CmaPrice.Trade;
var sampling = new Duration { Seconds = 1 * 60 };

#### 3.3 Time period delimitation

In [ ]:
var today = DateTime.Today;
var dateIntervals = new DateInterval
{
    StartDate = new Date { Year = 2020, Month = 02, Day = 18 },
    EndDate = new Date { Year = 2020, Month = 02, Day = 18 }
};

var timeInterval = new TimeInterval
{
    StartTime = new TimeOfDay { Hours = 08, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 }
};

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints();
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);   

The following code snippets create requests for: *Long SMA*, *Short SMA* and *Prices*:

In [ ]:
// Generate CMA request
var request = new TickCmaRequest()
{
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Field = field,
    Sampling = sampling
};

// Instantiate the tick CMA service
var service = new TickCmaService.TickCmaServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve CMA data

In [ ]:
var cmas = new List<TickCmaResponse>();

var call = service.TickCma(request, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    cmas.Add(data);
}
//display(cmas);

#### 4.4 Plot CMA and prices
Plot CMA and prices request' results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. Plot a simple line graph with the CMA data points and prices: 

In [ ]:
// Plot simultaneously prices and cumulative moving averages
var cmaPlots = new List<Scattergl>
{
    new Scattergl
    {
        name = "CMA",
        mode = "lines",
        x = cmas.Select(l => l.TimeStamp.ToDateTime()),
        y = cmas.Select(l => l.Average)
    },
    new Scattergl
    {
        name = "Prices",
        mode = "lines",
        x = cmas.Select(l => l.TimeStamp.ToDateTime()),
        y = cmas.Select(l => l.Value)
    }
};
var chart = Chart.Plot(cmaPlots);
chart.WithTitle($"CMA and prices for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);